In [ ]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
import os
from glob import glob
from PIL import Image

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical # convert to one-hot-encoding

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_dir='/content/drive/My Drive/archive'
df = pd.read_csv(os.path.join(data_dir, '/content/drive/MyDrive/Dataset/hmnist_28_28_RGB.csv'))

In [ ]:
ldf = pd.read_csv(os.path.join(data_dir, '/content/drive/MyDrive/Dataset/HAM10000_metadata.csv'))

In [ ]:
ldf.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
ldf['image_id'].shape

(10015,)

In [ ]:
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [ ]:
image_dir_part_1 = '/content/drive/MyDrive/Dataset/HAM10000/HAM10000_images_part_1'
image_dir_part_2 = '/content/drive/MyDrive/Dataset/HAM10000/HAM10000_images_part_2'

In [ ]:
import os
import numpy as np
from PIL import Image

def load_images_from_directory(directory, desired_height, desired_width):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img = Image.open(os.path.join(directory, filename))
            img = img.resize((desired_width, desired_height))  # Resize to desired dimensions
            img = np.array(img)
            images.append(img)
    return np.array(images)

# Set your desired image dimensions
desired_width = 256
desired_height = 192
channels = 3  # Assuming RGB images

# Load and resize images from both directories
images_part_1 = load_images_from_directory(image_dir_part_1, desired_height, desired_width)
images_part_2 = load_images_from_directory(image_dir_part_2, desired_height, desired_width)

# Stack the images into a single array with shape (no. of samples, height, width, channels)
all_images = np.vstack([images_part_1, images_part_2])

# Save the NumPy array as a .npy file if needed
np.save('all_images.npy', all_images)

In [ ]:
all_images = np.load('all_images.npy')

In [ ]:
all_images.shape

(10025, 192, 256, 3)

In [ ]:
all_images.shape, images_part_1.shape

((10025, 192, 256, 3), (5009, 192, 256, 3))

In [ ]:
images_part_2.shape

(5016, 192, 256, 3)

In [ ]:
lesion_danger = {
    'nv': np.array([1,0,0,0,0,0,0]),
    'mel': np.array([0,1,0,0,0,0,0]),
    'bkl': np.array([0,0,1,0,0,0,0]),
    'bcc': np.array([0,0,0,1,0,0,0]),
    'akiec': np.array([0,0,0,0,1,0,0]),
    'vasc': np.array([0,0,0,0,0,1,0]),
    'df': np.array([0,0,0,0,0,0,1])
}

ldf["dxn"] = ldf["dx"].map(lesion_danger.get)

In [ ]:
ldf['dxn'].shape

(10015,)

In [ ]:
import os
import numpy as np
from PIL import Image

image_dir_part_1 = '/content/drive/MyDrive/Dataset/HAM10000/HAM10000_images_part_1'
image_dir_part_2 = '/content/drive/MyDrive/Dataset/HAM10000/HAM10000_images_part_2'

def load_labels_from_directory(directory):
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
          labels.append(ldf.loc[ldf['image_id'] == filename[:12], 'dxn'].iloc[0])
    return labels

labels_1 = load_labels_from_directory(image_dir_part_1)
labels_2 = load_labels_from_directory(image_dir_part_2)

# Stack the images into a single array with shape (no. of samples, height, width, channels)
all_labels = np.concatenate([labels_1, labels_2])

# Save the NumPy array as a .npy file if needed
np.save('all_labels.npy', all_labels)

In [ ]:
all_labels = np.load('all_labels.npy')

In [ ]:
all_labels.shape

(10025, 7)

In [ ]:
labels = all_labels
labels.shape

(10025, 7)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np

# Specify the full file path to all_samples.npy
file_path = '/content/all_images.npy'

# Load the data from the specified file path
data = np.load(file_path)


# Load the data from all_samples.npy
labels_path='/content/all_labels.npy'

# Load labels or classes from a corresponding file (e.g., labels.npy)
labels = np.load(labels_path)

ndata = data[:1000]
nlabels = labels[:1000]
# Split the data and labels into training and testing sets
test_size = 0.2  # You can adjust the test size as needed
x_train, x_test, y_train, y_test = train_test_split(ndata, nlabels,test_size=test_size, random_state=42)

# Now x_train, x_test, y_train, and y_test contain the training and testing data and labels.

In [ ]:
x_train.shape, x_test.shape,y_train.shape, y_test.shape

((800, 192, 256, 3), (200, 192, 256, 3), (800, 7), (200, 7))

In [ ]:
np.save('x_train.npy',x_train)
np.save('x_test.npy',x_test)
np.save('y_train.npy',y_train)
np.save('y_test.npy',y_test)

In [ ]:
x_test=np.load("/content/x_test.npy")
y_test=np.load("/content/y_test.npy")
x_train=np.load("/content/x_train.npy")
y_train=np.load("/content/y_train.npy")